# Semantic Search on PDF Documents with qFlat Index

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

This example demonstrates how to use KDB.AI to run semantic search on unstructured text documents.

<div class="alert alert-block alert-info">
<b>Tip:</b> This sample uses ‘qFlat’ , a new vector index choice in KDB.AI. It will support the same API options as the existing ‘Flat’ index but with the significant difference that the index is stored on-disk and memory-mapped as required. This means data inserts will have negligible memory and cpu footprints. The vector index can grow and be searched as long as there is disk space available and works great for datasets with up to 1,000,000 vectors. Among other cases, this stands out as a great index for memory contrained situations such as edge devices.
</div>

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

### Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

0. Setup
1. Load PDF Data
2. KDB.AI Table Creation
3. LlamaIndex index & query_engine setup
4. Retrieve Similar Sentences & RAG
5. Delete the KDB.AI Table

---

## 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

### Set Environment Variables

In [ ]:
!pip install llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-readers-file llama-index-vector-stores-kdbai
!pip install kdbai_client
!pip install pandas


### Import Packages

In [49]:

import os
from getpass import getpass
import re
import os
import shutil
import time
import urllib

import pandas as pd

from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.kdbai import KDBAIVectorStore

import kdbai_client as kdbai

OUTDIR = "pdf"
RESET = True

In [50]:
### !!! Only run this cell if you need to download the data into your environment, for example in Colab
### This downloads research paper pdf into your environment
if os.path.exists("./data/research_paper.pdf") == False:
  !mkdir ./data
  !wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf

In [51]:
# OpenAI API Key: https://platform.openai.com/api
os.environ["OPENAI_API_KEY"] = (
    os.environ["OPENAI_API_KEY"]
    if "OPENAI_API_KEY" in os.environ
    else getpass("OpenAI API Key: ")
)

In [52]:
# Set up LlamaIndex Parameters

import nest_asyncio

nest_asyncio.apply()

EMBEDDING_MODEL  = "text-embedding-3-small"
GENERATION_MODEL = 'gpt-4o-mini'

llm = OpenAI(model=GENERATION_MODEL)
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)

Settings.llm = llm
Settings.embed_model = embed_model

### Configure Console

In [53]:
pd.set_option("display.max_colwidth", 300)

### Define Helper Functions

In [54]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

## 1. Load PDF Data

### Read Text From PDF Document

We LlamaIndex SimpleDirectorReader to read in our PDF file.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way.

In [55]:
reader = SimpleDirectoryReader(
    input_dir="data",
)
documents = reader.load_data()

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [56]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [ ]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [58]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have two columns.

At this point you will select the index and metric you want to use for searching.

In this case, we will use the qFlat index, Euclidean Distance (L2) for the search metric, and we specify the number of dimensions of our embeddings (384).

In [59]:
#Set up the schema and indexes for KDB.AI table, specifying embeddings column with 384 dimensions, Euclidean Distance, and flat index
pdf_schema = [
    {"name": "document_id", "type": "bytes"},
    {"name": "text", "type": "bytes"},
    {"name": "embeddings", "type": "float64s"}
]

indexes = [
    {
        "name": "qflat_index",
        "type": "qFlat",
        "column": "embeddings",
        "params": {"dims": 1536, "metric": "L2"},
    }
]


### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [60]:
# get the database connection. Default database name is 'default'
database = session.database('default')

# First ensure the table does not already exist
try:
    database.table("pdf").drop()
except kdbai.KDBAIException:
    pass

In [61]:
table = database.create_table("pdf", pdf_schema, indexes=indexes)

We can use `query` to see our table exists but is empty.

In [ ]:
table.query()

## 3. LlamaIndex index & query_engine setup
Define the index: using KDB.AI as the vector store, chunk, embed, and load the document into KDB.AI

In [63]:
vector_store = KDBAIVectorStore(table)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=1000, chunk_overlap=0)],
)

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [ ]:
show_df(table.query())

#### Set up the LlamaIndex Query Engine

In [65]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    vector_store_kwargs={
                    "index" : "qflat_index",
                },
)

## 4. Retrieve Similar Sentences & RAG



Now that the embeddings are stored in KDB.AI, we can perform semantic search using through the LlamaIndex query engine.

### Search 1


In [66]:
search_term1 = "number of interstellar objects in the milky way"

In [ ]:
retrieved_chunks = query_engine.retrieve(search_term1)
print(retrieved_chunks)
for i in retrieved_chunks:
    print(i.node.get_text())
    print("____________________")

We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way.

### Now we can perform RAG, passing the retrieved chunks from above to the LLM for a generate response:

In [ ]:
result = query_engine.query(search_term1)
print(result.response)

### Search 2

Let's try another search term.

In [22]:
search_term2 = "how does planet formation occur"

In [ ]:
retrieved_chunks = query_engine.retrieve(search_term2)
for i in retrieved_chunks:
    print(i.node.get_text())
    print("____________________")

Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way.

In [ ]:
result = query_engine.query(search_term2)
print(result.response)

## 5. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [25]:
table.drop()

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/ejgOzTpo)